<a href="https://colab.research.google.com/github/baquer/FacQA/blob/master/FacQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step-1:
Download Dependencies

In [1]:
!python -m nltk.downloader punkt

/usr/lib/python3.8/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 103.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.7 MB/s eta 0:00:00


In [3]:
!git clone https://github.com/patil-suraj/question_generation.git

Cloning into 'question_generation'...
remote: Enumerating objects: 268, done.
remote: Total 268 (delta 0), reused 0 (delta 0), pack-reused 268
Receiving objects: 100% (268/268), 299.04 KiB | 19.94 MiB/s, done.
Resolving deltas: 100% (140/140), done.


In [4]:
%cd question_generation

/content/question_generation


In [5]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.0 MB/s eta 0:00:00


In [6]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=22f557e69c053df0a1a2862013f78c7496ad782b8c46367cfc081c7087d5c2d2
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


## Step-2:
Import Libraries

In [7]:
import warnings
import pandas as pd
# ignore all warnings
warnings.filterwarnings("ignore")

from flask import Flask, render_template, jsonify
from flask import request, url_for, redirect, session
import requests
import html5lib
from bs4 import BeautifulSoup
# from pipelines import pipeline

from transformers import pipeline as p
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import corpus_bleu
import torch

from sentence_transformers import SentenceTransformer
from googlesearch import search

Edited version of Pipeline.py from https://github.com/patil-suraj/question_generation.git

In [8]:
import itertools
import logging
from typing import Optional, Dict, Union

from nltk import sent_tokenize

import torch
from transformers import(
    AutoModelForSeq2SeqLM, 
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
)

logger = logging.getLogger(__name__)

class QGPipeline:
    """Poor man's QG pipeline"""
    def __init__(
        self,
        model: PreTrainedModel,
        tokenizer: PreTrainedTokenizer,
        ans_model: PreTrainedModel,
        ans_tokenizer: PreTrainedTokenizer,
        qg_format: str,
        use_cuda: bool
    ):
        self.model = model
        self.tokenizer = tokenizer

        self.ans_model = ans_model
        self.ans_tokenizer = ans_tokenizer

        self.qg_format = qg_format

        self.device = "cuda" if torch.cuda.is_available() and use_cuda else "cpu"
        self.model.to(self.device)

        if self.ans_model is not self.model:
            self.ans_model.to(self.device)

        assert self.model.__class__.__name__ in ["T5ForConditionalGeneration", "BartForConditionalGeneration"]
        
        if "T5ForConditionalGeneration" in self.model.__class__.__name__:
            self.model_type = "t5"
        else:
            self.model_type = "bart"

    def __call__(self, inputs: str):
        inputs = " ".join(inputs.split())
        sents, answers = self._extract_answers(inputs)
        flat_answers = list(itertools.chain(*answers))
        
        if len(flat_answers) == 0:
          return []

        if self.qg_format == "prepend":
            qg_examples = self._prepare_inputs_for_qg_from_answers_prepend(inputs, answers)
        else:
            qg_examples = self._prepare_inputs_for_qg_from_answers_hl(sents, answers)
        
        qg_inputs = [example['source_text'] for example in qg_examples]
        questions = self._generate_questions(qg_inputs)
        output = [{'answer': example['answer'], 'question': que} for example, que in zip(qg_examples, questions)]
        return output
    
    def _generate_questions(self, inputs):
        inputs = self._tokenize(inputs, padding=True, truncation=True)
        
        outs = self.model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device), 
            max_length=32,
            num_beams=4,
        )
        
        questions = [self.tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
        return questions
    
    def _extract_answers(self, context):
        sents, inputs = self._prepare_inputs_for_ans_extraction(context)
        inputs = self._tokenize(inputs, padding=True, truncation=True)

        outs = self.ans_model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device), 
            max_length=32,
        )
        
        dec = [self.ans_tokenizer.decode(ids, skip_special_tokens=False) for ids in outs]
        answers = [item.split('<sep>') for item in dec]
        answers = [i[:-1] for i in answers]
        
        return sents, answers
    
    def _tokenize(self,
        inputs,
        padding=True,
        truncation=True,
        add_special_tokens=True,
        max_length=512
    ):
        inputs = self.tokenizer.batch_encode_plus(
            inputs, 
            max_length=max_length,
            add_special_tokens=add_special_tokens,
            truncation=truncation,
            padding="max_length" if padding else False,
            pad_to_max_length=padding,
            return_tensors="pt"
        )
        return inputs
    
    def _prepare_inputs_for_ans_extraction(self, text):
        sents = sent_tokenize(text)

        inputs = []
        for i in range(len(sents)):
            source_text = "extract answers:"
            for j, sent in enumerate(sents):
                if i == j:
                    sent = "<hl> %s <hl>" % sent
                source_text = "%s %s" % (source_text, sent)
                source_text = source_text.strip()
            
            if self.model_type == "t5":
                source_text = source_text + " </s>"
            inputs.append(source_text)

        return sents, inputs
    
    def _prepare_inputs_for_qg_from_answers_hl(self, sents, answers):
        # print(sents)
        # print(answers)
        # print("hello")
        inputs = []
        for i, answer in enumerate(answers):
            if len(answer) == 0: continue
            # print(i)
            # print(answers)
            for answer_text in answer:
                # print(answer_text)
                sent = sents[i]
                # print(answer_text)
                # print(sent)
                # print(sents)
                sents_copy = sents[:]
                
                answer_text = answer_text.strip()
                # print(answer_text)
                if '<pad>' in answer_text:
                    answer_text = answer_text[6:]
                # answer_text = answer_text.split()[0]
                # print(answer_text)
                # print(sent)
                # # print(sent.index("a photo"))
                #
                # print(sent.index(answer_text))
                try:
                    ans_start_idx = sent.index(answer_text)
                except:
                    continue
                
                sent = f"{sent[:ans_start_idx]} <hl> {answer_text} <hl> {sent[ans_start_idx + len(answer_text): ]}"
                sents_copy[i] = sent
                
                source_text = " ".join(sents_copy)
                source_text = f"generate question: {source_text}" 
                if self.model_type == "t5":
                    source_text = source_text + " </s>"
                
                inputs.append({"answer": answer_text, "source_text": source_text})
        
        return inputs
    
    def _prepare_inputs_for_qg_from_answers_prepend(self, context, answers):
        flat_answers = list(itertools.chain(*answers))
        examples = []
        for answer in flat_answers:
            source_text = f"answer: {answer} context: {context}"
            if self.model_type == "t5":
                source_text = source_text + " </s>"
            
            examples.append({"answer": answer, "source_text": source_text})
        return examples

    
class MultiTaskQAQGPipeline(QGPipeline):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def __call__(self, inputs: Union[Dict, str]):
        if type(inputs) is str:
            # do qg
            return super().__call__(inputs)
        else:
            # do qa
            return self._extract_answer(inputs["question"], inputs["context"])
    
    def _prepare_inputs_for_qa(self, question, context):
        source_text = f"question: {question}  context: {context}"
        if self.model_type == "t5":
            source_text = source_text + " </s>"
        return  source_text
    
    def _extract_answer(self, question, context):
        source_text = self._prepare_inputs_for_qa(question, context)
        inputs = self._tokenize([source_text], padding=False)
    
        outs = self.model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device), 
            max_length=16,
        )

        answer = self.tokenizer.decode(outs[0], skip_special_tokens=True)
        return answer


class E2EQGPipeline:
    def __init__(
        self,
        model: PreTrainedModel,
        tokenizer: PreTrainedTokenizer,
        use_cuda: bool
    ) :

        self.model = model
        self.tokenizer = tokenizer

        self.device = "cuda" if torch.cuda.is_available() and use_cuda else "cpu"
        self.model.to(self.device)

        assert self.model.__class__.__name__ in ["T5ForConditionalGeneration", "BartForConditionalGeneration"]
        
        if "T5ForConditionalGeneration" in self.model.__class__.__name__:
            self.model_type = "t5"
        else:
            self.model_type = "bart"
        
        self.default_generate_kwargs = {
            "max_length": 256,
            "num_beams": 4,
            "length_penalty": 1.5,
            "no_repeat_ngram_size": 3,
            "early_stopping": True,
        }
    
    def __call__(self, context: str, **generate_kwargs):
        inputs = self._prepare_inputs_for_e2e_qg(context)

        # TODO: when overrding default_generate_kwargs all other arguments need to be passsed
        # find a better way to do this
        if not generate_kwargs:
            generate_kwargs = self.default_generate_kwargs
        
        input_length = inputs["input_ids"].shape[-1]
        
        # max_length = generate_kwargs.get("max_length", 256)
        # if input_length < max_length:
        #     logger.warning(
        #         "Your max_length is set to {}, but you input_length is only {}. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)".format(
        #             max_length, input_length
        #         )
        #     )

        outs = self.model.generate(
            input_ids=inputs['input_ids'].to(self.device), 
            attention_mask=inputs['attention_mask'].to(self.device),
            **generate_kwargs
        )

        prediction = self.tokenizer.decode(outs[0], skip_special_tokens=True)
        questions = prediction.split("<sep>")
        questions = [question.strip() for question in questions[:-1]]
        return questions
    
    def _prepare_inputs_for_e2e_qg(self, context):
        source_text = f"generate questions: {context}"
        if self.model_type == "t5":
            source_text = source_text + " </s>"
        
        inputs = self._tokenize([source_text], padding=False)
        return inputs
    
    def _tokenize(
        self,
        inputs,
        padding=True,
        truncation=True,
        add_special_tokens=True,
        max_length=512
    ):
        inputs = self.tokenizer.batch_encode_plus(
            inputs, 
            max_length=max_length,
            add_special_tokens=add_special_tokens,
            truncation=truncation,
            padding="max_length" if padding else False,
            pad_to_max_length=padding,
            return_tensors="pt"
        )
        return inputs


SUPPORTED_TASKS = {
    "question-generation": {
        "impl": QGPipeline,
        "default": {
            "model": "valhalla/t5-small-qg-hl",
            "ans_model": "valhalla/t5-small-qa-qg-hl",
        }
    },
    "multitask-qa-qg": {
        "impl": MultiTaskQAQGPipeline,
        "default": {
            "model": "valhalla/t5-small-qa-qg-hl",
        }
    },
    "e2e-qg": {
        "impl": E2EQGPipeline,
        "default": {
            "model": "valhalla/t5-small-e2e-qg",
        }
    }
}

def pipeline(
    task: str,
    model: Optional = None,
    tokenizer: Optional[Union[str, PreTrainedTokenizer]] = None,
    qg_format: Optional[str] = "highlight",
    ans_model: Optional = None,
    ans_tokenizer: Optional[Union[str, PreTrainedTokenizer]] = None,
    use_cuda: Optional[bool] = True,
    **kwargs,
):
    # Retrieve the task
    if task not in SUPPORTED_TASKS:
        raise KeyError("Unknown task {}, available tasks are {}".format(task, list(SUPPORTED_TASKS.keys())))

    targeted_task = SUPPORTED_TASKS[task]
    task_class = targeted_task["impl"]

    # Use default model/config/tokenizer for the task if no model is provided
    if model is None:
        model = targeted_task["default"]["model"]
    
    # Try to infer tokenizer from model or config name (if provided as str)
    if tokenizer is None:
        if isinstance(model, str):
            tokenizer = model
        else:
            # Impossible to guest what is the right tokenizer here
            raise Exception(
                "Impossible to guess which tokenizer to use. "
                "Please provided a PretrainedTokenizer class or a path/identifier to a pretrained tokenizer."
            )
    
    # Instantiate tokenizer if needed
    if isinstance(tokenizer, (str, tuple)):
        if isinstance(tokenizer, tuple):
            # For tuple we have (tokenizer name, {kwargs})
            tokenizer = AutoTokenizer.from_pretrained(tokenizer[0], **tokenizer[1])
        else:
            tokenizer = AutoTokenizer.from_pretrained(tokenizer)
    
    # Instantiate model if needed
    if isinstance(model, str):
        model = AutoModelForSeq2SeqLM.from_pretrained(model)
    
    if task == "question-generation":
        if ans_model is None:
            # load default ans model
            ans_model = targeted_task["default"]["ans_model"]
            ans_tokenizer = AutoTokenizer.from_pretrained(ans_model)
            ans_model = AutoModelForSeq2SeqLM.from_pretrained(ans_model)
        else:
            # Try to infer tokenizer from model or config name (if provided as str)
            if ans_tokenizer is None:
                if isinstance(ans_model, str):
                    ans_tokenizer = ans_model
                else:
                    # Impossible to guest what is the right tokenizer here
                    raise Exception(
                        "Impossible to guess which tokenizer to use. "
                        "Please provided a PretrainedTokenizer class or a path/identifier to a pretrained tokenizer."
                    )
            
            # Instantiate tokenizer if needed
            if isinstance(ans_tokenizer, (str, tuple)):
                if isinstance(ans_tokenizer, tuple):
                    # For tuple we have (tokenizer name, {kwargs})
                    ans_tokenizer = AutoTokenizer.from_pretrained(ans_tokenizer[0], **ans_tokenizer[1])
                else:
                    ans_tokenizer = AutoTokenizer.from_pretrained(ans_tokenizer)

            if isinstance(ans_model, str):
                ans_model = AutoModelForSeq2SeqLM.from_pretrained(ans_model)
    
    if task == "e2e-qg":
        return task_class(model=model, tokenizer=tokenizer, use_cuda=use_cuda)
    elif task == "question-generation":
        return task_class(model=model, tokenizer=tokenizer, ans_model=ans_model, ans_tokenizer=ans_tokenizer, qg_format=qg_format, use_cuda=use_cuda)
    else:
        return task_class(model=model, tokenizer=tokenizer, ans_model=model, ans_tokenizer=tokenizer, qg_format=qg_format, use_cuda=use_cuda)

## Main Code:

In [9]:
# app = Flask(__name__)

# global lp
# lp = {}
ques_ans = {}
majority_l = {}
# @app.route('/')
# def hello_world():
#     return render_template('index.html')

# @app.route('/predict',methods=['POST'])
def predict(head,text):
    # ll = [x for x in request.form.values()]
    # head = ll[0]
    # text = ll[1]
    # session["head"] = head
    # session['text'] = text
    # return redirect(url_for('ques_ans', text=text))

    # print(text)
    link_list = take_text(head)
    # print(link_list)
    nlp = pipeline("multitask-qa-qg")
    question_ans = nlp(text)
    only_question = []
    ques_ans_dict = {}
    ans_all = []
    for dictionary in question_ans:
        only_question.append(dictionary['question'])
        ques_ans_dict[dictionary['question']] = dictionary['answer']
    # print(ques_ans_dict)
    # print(only_question)
    lp=ques_ans_dict
    query_search_text(link_list, only_question, ans_all)
    # print(ans_all)
    dict_ans = []
    get_answers(ques_ans_dict, dict_ans)
    maj_list = getMajority(ans_all)
    # print(maj_list)
    finalList = []
    importf(dict_ans, maj_list)
    verdict = metrics(dict_ans, maj_list, finalList)
    verdict2 = metrics2(dict_ans, maj_list)
    verdict3 = sbertSimilarity(dict_ans, maj_list)
    score = round(verdict3 * 100, 2)
    # l = [score,ques_ans_dict,maj_list]
    # return redirect(url_for('ques_ans', score=l))
    # if request.method == 'POST':
    #     date = request.form.get('ques')
    #     return redirect(url_for('ques_ans', ques=date))
    # print(request.method)
    # if score >= 50:
    #     return render_template('index.html', prediction_text='This Article is Trustworthy with a S-Bert Score of {}'.format(score)
    # else:
    #     return render_template('index.html', prediction_text='This Article is Fake with a S-Bert Score of {}'.format(score))
    return score
    # print(request.method)
    # if request.method == 'POST':
    #     date = request.form.get('date')
    #     return redirect(url_for('booking', date=date))
#
# @app.route('/ques_ans/<score>')
# def ques_ans(score):
#     mylist = print_ques_ans()
#     print(request.method)
#     print(lp)
#     # head = session.get("head")
#     # text = session.get("text")
#     print(score)
#     r = score[1:6]
#     q = score[10:60]
#     return render_template('table.html', prediction_text = r, question_answer=q)


def take_text(head):
    try:
        from googlesearch import search
    except ImportError:
        print("No module named 'google' found")
    # print(head)
    query = head.split("\n")[0]
    # print(query)
    link_list = []
    for j in search(query):
        link_list.append(j)

    #link_list = ['https://www.newsweek.com/pope-francis-trump-immigration-jerusalem-christmas-758710','https://www.reuters.com/article/christmas-season-pope-urbi-et-orbi-idAFL8N1OP0DJ',]

    return link_list


def query_search_text(link_list, only_question, ans_all):
    new_list = link_list[0: 2]
    for link in new_list:
        # print(link)
        URL = link
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}
        #session = requests.Session()
        r = requests.get(URL, headers=headers)
        soup = BeautifulSoup(r.content, 'html5lib')
        tags = soup.find_all('h1')
        #
        heading = ""
        i = 0
        for t in tags:
            try:
                if i == 0:
                    heading = t.text
                    i += 1
                # print(t.text)
            except:
                continue

        tags1 = soup.find_all('p')
        para = ""
        i = 0
        for t in tags1:
            try:
                # print(t.text)
                para += t.text
            except:
                continue
        answer_from_link(para, only_question, ans_all)

def answer_from_link(para, only_question, ans_obtained_all):
    # print(para)
    with torch.no_grad():
        question_answerer = p("question-answering", model='distilbert-base-cased-distilled-squad')
        context = para
        ans = []
        for question in only_question:
            result = question_answerer(question=question, context=context)
            ans.append(result['answer'])
        ans_obtained_all.append(ans)


def most_frequent(List):
    counter = 0
    num = List[0]

    for i in List:
        curr_frequency = List.count(i)
        if (curr_frequency > counter):
            counter = curr_frequency
            num = i

    return num

def getMajority(ans):
  n = len(ans)
  m = len(ans[0])
  final = []
  for i in range(0, m):
    temp = []
    for j in range(0, n):
      temp.append(ans[j][i])
    element = most_frequent(temp)
    final.append(element)
  return final

def metrics(list1, list2, newList):
  n = len(list1)
  m = len(list2)
  # print(n)
  # print(m)
  i = 0
  j = 0
  matching = 0
  non_matching = 0
  while (i < n and j < m):
      if (list1[i] == list2[j]):
          matching += 1
          newList.append(list1[i])
      else:
          non_matching += 1
          newList.append(list2[i])
      i += 1
      j += 1
  score = (matching / len(list1)) * 100
  if matching >= non_matching:
      return "Claim is true" + str(score)
  else :
      return "Claim is false" + str(score)

def metrics2(ans_list, maj_list):
    score = sentence_bleu([ans_list], maj_list, weights=(1, 0, 0, 0))
    # print(score)
    if score > 0.7:
        return "Article is Trustworthy and It's Score is" + str(score)
    else:
        return "Article is Fake and It's Score is" + str(score)


def get_answers(dict, ans):
    for ques in dict:
        ans.append(dict[ques])

def sbertSimilarity(ans, maj):
    sentence_transformer_model = SentenceTransformer('bert-base-nli-mean-tokens')
    # print("yes")
    sentence_embeddings = sentence_transformer_model.encode(ans)
    sentence_embeddings2 = sentence_transformer_model.encode(maj)
    # print("yes")
    cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    output = torch.mean(cos(torch.tensor(sentence_embeddings), torch.tensor(sentence_embeddings2))).item()
    # print(output)
    return output

def importf(dict, list):
    ques_ans = dict
    majority_l = list

def print_ques_ans():
    print_list = []
    # for key in ques_ans:
    #     print_list.append("Ques:- " + key + " ans:- " + ques_ans[key])
    for i in majority_l:
        print_list.append(" obtained ans:- " + i)
    return print_list


# if __name__ == '__main__':
#     app.run(debug=True)

## Step-3: 
Input the Headline of the News article in head variable and body in text variable 

TRUE ARTICLE EXAMPLE

In [10]:
head = "Silicon Valley blasts Senate proposal to tax startup options"
text = "SAN FRANCISCO (Reuters) - A proposal by the U.S. Senate to change the way shares in startup companies are taxed incited panic and dread in Silicon Valley on Monday, with startup founders and investors warning of nothing less than the demise of their industry should the proposal become law. The provision in the Senate’s tax reform plan, which appeared to catch the industry by surprise, involves the treatment of employee stock options. These options give the holder the right to purchase shares in the future at a set price and can be very valuable if a company does well and the share price increases. Options are often a major portion of the compensation for  startup employees and founders, who take lower salaries in anticipation of a big payout if their startup takes off. Options typically vest over a four-year period. Senate Republicans have now proposed taxing those stock options as they vest and before startup employees have the opportunity to cash them in, resulting in annual tax bills that could easily climb into the tens of thousands of dollars, say startup founders and venture capitalists. “If there were a single piece of legislation to adversely affect startups, it would be this,” said Venky Ganesan, managing director at venture capital firm Menlo Ventures. “Everyone is freaked out.” Justin Field, vice president of government affairs at the National Venture Capital Association, said that the Senate’s proposed tax change would be “crippling” to the startup industry. How far the provision gets remains to be seen. The National Venture Capital Association was successful in getting a similar proposal removed from the House tax bill, although it “didn’t fully appreciate” the Senate’s intention to add the tax provision, Field said. The association also helped to steer lawmakers away from a proposal discussed late last year to tax venture capitalists’ profits on investments at a higher rate. Republican Senator Rob Portman of Ohio, a member of the Senate Committee on Finance, has filed an amendment to repeal the provision in the tax bill, according to his spokesman.  Under current tax code, employees are taxed only when they exercise their options. Options are exercised when the price they were granted at—known as the strike price—is lower than the share price, and some shares can then be sold to pay the taxes. But the Senate proposal would require startup employees to pay regular income tax on the value gain of their stock options even before they are exercised. These options are illiquid assets, and cannot be spent or saved. “What this would mean is every month, when your equity compensation vests a little bit, you will owe taxes on it even though you can’t do anything with that equity compensation,” Fred Wilson, a venture capitalist with Union Square Ventures, wrote on his blog Monday. For instance, if a startup employee receives stock options at a dollar per share, and the shares increase in value by $1 every year during the four-year vesting period, the employee would have to pay income tax on $1 per share after the first year, pay again on the $1 increase in value after the second year, and so on. When that employee owns hundreds of thousands and even millions of shares, that is a hefty bill to pay. And there is always the risk the startup will eventually fail. “This reform will force the average employee to pay taxes on that bet well before they even know if it’s a winning ticket,” said Amanda Kahlow, founder and executive chairman of marketing data startup 6sense. For startup founders in particular, such a tax bill could be ruinous.  “It would mean that I would have to sell the company,” said Shoaib Makani, founder and chief executive of long-haul trucking startup KeepTruckin. “I have zero net worth aside from the common stock I hold in the company. It would be impossible. I would be in default.” Some executives in the startup industry, however, have pushed for companies to move toward bigger salaries so employees are not so dependent on options to buy a house or pay for other large expenses. And when startups suffer valuation cuts, employees can end up with worthless options. The Senate’s proposal came as a revenue-generating measure to help offset tax breaks in the bill. A spokesman for Senator Orrin Hatch, a Republican and chairman of the Senate Committee on Finance, did not respond to requests for comment and other Republicans on the committee were not immediately available.  A spokeswoman for Senator Ron Wyden, the committee’s ranking member and a Democrat, said he was aware of concerns that the provision would limit startups’ ability to attract talent."

## Step-4:
Call the main function to get the score for the News-Article

In [11]:
score = predict(head,text)
print('The Similarity/S-Bert score for the Input News Article is: ',score)

The Similarity/S-Bert score for the Input News Article is:  86.76


In [12]:
if score > 70:
  print('The Input News Article is Trustworthy')
else:
  print('The Input News Article is Fake')

The Input News Article is Trustworthy


FALSE ARTICLE EXAMPLE

In [13]:
head = "Watch: Trump Supporting Coal CEO Upset Trump Is Wiping Out ‘Thousands’ Of Coal Mining Jobs"
text = "While on the campaign trail, Donald Trump promised to revive the coal industry and after he took power, he signed an executive order rolling back a temporary ban on mining coal and a stream protection rule that was imposed by the Obama administration. Trump vowed to bring back thousands of mining jobs in struggling rural areas.  I made them this promise,  Trump said,  we will put our miners back to work. That was then and this is now.Coal CEO Robert Murray, a fierce supporter of Trump s, warns that if the Senate version of tax reform is enacted by the current occupant of the White House, he will be destroying thousands of coal mining jobs in the process.Murray said the Senate legislation will raise Murray Energy s taxes by $60 million a year,  notwithstanding the other so-called benefits the Senate has proposed,  CNBC reports. This means that very capital-intensive, highly leveraged employers, like coal-mining companies, will be forced out of business, with tragic consequences for our families and for many regions of our country,  Murray said. We won t have enough cash flow to exist. It wipes us out,  Murray told CNNMoney in an interview on Tuesday. This wipes out everything that President Trump has done for coal,  said Murray, the head of one of America s largest coal companies.CNN reports:Murray warned that a bankruptcy of his Ohio-based company would hurt its 5,500 employees along with their families. Asked if other coal mining companies could go out of business, he said:  Most certainly. Watch:Murray hosted a fundraiser for Trump during the campaign. And today, Murray warns that the GOP tax plan will  wipe out  coal mining jobs.Who saw this coming? Besides all of us. Trump supporters have been suckered.Image via screen capture. "

In [14]:
score = predict(head,text)
print('The Similarity/S-Bert score for the Input News Article is: ',score)

The Similarity/S-Bert score for the Input News Article is:  46.97


In [15]:
if score > 70:
  print('The Input News Article is Trustworthy')
else:
  print('The Input News Article is Fake')

The Input News Article is Fake
